In [1]:
import json
import sys
import math
from tqdm import tqdm
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import jieba
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/feiluhuan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/feiluhuan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
review_path = 'train_data/before_comment.csv'
review_df = pd.read_csv(review_path)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
review_df = review_df.iloc[:,1:]
review_df = review_df.dropna(axis=0,how='any')
review_df

,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park
0,xXgE_rj-tK9v_MIH3mXpaQ,3W-lOOaIVQZONZP83kab6g,Reggie and company cook up the real deal at th...,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,8CSdj8LF593qwMcduzHglQ,3W-lOOaIVQZONZP83kab6g,Convential Wisdom is that this is the best res...,5.0,3.5,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
2,JjXEPAR_tf3VCdngVS9GMg,3W-lOOaIVQZONZP83kab6g,Finally an organic grocery store in East Nashv...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,cBHzDxKCsosdDI4UR3w-eg,3W-lOOaIVQZONZP83kab6g,"This converted laundromat is a small, trendy s...",3.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
4,tU692E8N0xBQ7Ogc78gN2g,3W-lOOaIVQZONZP83kab6g,This is a great place to meet all sorts of peo...,3.0,4.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
588250,wbaeORfUCcxkwdh7mNapRg,8j1OffLqKtetcAG3xv3f0Q,We had such a good time! Scott Hinds was very ...,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
588251,cJGdlK7KeE78Qa0FydikIw,VTfl9PALOCiGV8SUBpCZEQ,We had a great buying experience. Michael Harr...,5.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
588252,U7esGceQIZ_1IG6Ib0c1lQ,m0-9woIKbK-R4F8RVEe4BQ,My colleague ordered on my behalf and she came...,5.0,2.5,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
588253,Y4uWSRQ-GyOaxA2c-qZxWw,QTVdM8ThpshrX-8GTcdk2w,Love this place! Great for toys and maternity...,5.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [4]:
# 分train和test
user_groups = dict(list(review_df.groupby('user_id')))
ct = 0
for user, df in tqdm(user_groups.items()):
    if df.shape[0] > 2:
        if ct == 0:
            all_train = df.iloc[:math.ceil(df.shape[0]*0.6), :]
            all_test = df.iloc[math.ceil(df.shape[0]*0.6):, :]
        else:
            all_train = pd.concat([all_train, df.iloc[:math.ceil(df.shape[0]*0.6), :]])
            all_test = pd.concat([all_test, df.iloc[math.ceil(df.shape[0]*0.6):, :]])
        ct += 1
all_train

100%|██████████| 215518/215518 [41:39<00:00, 86.22it/s] 


,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park
108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,Went for a nice lunch because prices in the Ga...,4.0,2.5,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0
108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,Ate dinner there tonight. Service was pretty f...,4.0,3.5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,This is the best Japanese food I've had in Nas...,5.0,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,Ok so this is the best Chinese food I've had i...,5.0,3.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0
331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,I love this place I got my lashes done by Nina...,5.0,3.5,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87906,hn3Rg2JrhQoDJBEhrpuwWg,zzz-M4QvkEpUWWPL9RTzLA,We have been twice and LOVED it the first time...,2.0,3.5,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
153153,Y989Hsw30AYUMcyNjBz9iQ,zzz-M4QvkEpUWWPL9RTzLA,We went here last Friday night with friends an...,2.0,3.5,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
153155,dNmqRPHBys4dWH4nReO7MA,zzz-M4QvkEpUWWPL9RTzLA,"Based on the website, I was expecting somethin...",1.0,3.5,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
153392,qO-r-NipYXcuVXjIbaglEQ,zzz-M4QvkEpUWWPL9RTzLA,I got my haircut on Saturday and I've had at l...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
all_test

,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,Smoking,Park
108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,The place is absolutely huge! That's the attra...,4.0,3.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,Ok guys this is one of my favorite vegan spots...,5.0,4.5,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0
533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,I love this place I tried the Cuban sandwich a...,5.0,4.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,It's greasy but what do you expect. It is good...,3.0,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0
394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,Definitely gets busy so reserve spot in line o...,5.0,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357236,d59JUaCVMZj-ZEtlWcOKZw,zzsPf8xNW11nd0B6MZqfRw,Awesome unique food and a great addition to do...,5.0,4.5,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0
335253,nKJ6RnjIVfWrXO2DUrBgCw,zzz-M4QvkEpUWWPL9RTzLA,I ordered the fish tacos the other night and t...,1.0,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
370287,8BEIpKZu-ddcptxDbqMwLQ,zzz-M4QvkEpUWWPL9RTzLA,"I went here for the first time this morning, a...",4.0,3.5,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
390130,8WNQxb0CiROek9SW7PopDQ,zzz-M4QvkEpUWWPL9RTzLA,What a horrible experience. Either the waitres...,1.0,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0


In [8]:
# training set利用LDA
punctuations = [',', '.', ':', ';', '?', '(', ')', '[', ']', '&', '!', '*', '@', '#', '$', '%']
corpus = []
for sent in all_train['text']:
    sent = sent.lower()
    words = word_tokenize(sent)
    words = [word for word in words if word not in punctuations]
    stopWords = set(stopwords.words('english'))
    words = [word for word in words if word not in stopWords]
    ps = PorterStemmer()
    words = [ps.stem(word) for word in words]
    corpus.append(' '.join(words))
    
cntVector = CountVectorizer(stop_words='english')
cntTf = cntVector.fit_transform(corpus)

lda = LatentDirichletAllocation(n_components=100, learning_offset=50, learning_method='online')
# 每条comment属于每一类的概率
probs = lda.fit_transform(cntTf)
probs

array([[0.00020408, 0.00020408, 0.00020408, ..., 0.00020408, 0.00020408,
        0.00020408],
       [0.00018868, 0.00018868, 0.00018868, ..., 0.00018868, 0.00018868,
        0.00018868],
       [0.00052632, 0.00052632, 0.00052632, ..., 0.00052632, 0.00052632,
        0.00052632],
       ...,
       [0.00037037, 0.00037037, 0.00037037, ..., 0.00037037, 0.00037037,
        0.00037037],
       [0.00038462, 0.00038462, 0.00038462, ..., 0.00038462, 0.00038462,
        0.00038462],
       [0.0003125 , 0.0003125 , 0.0003125 , ..., 0.0315625 , 0.0315625 ,
        0.0003125 ]])

In [10]:
for i in range(100):
    name = 'topic_' + str(i+1)
    all_train[name] = probs[:,i]
all_train

,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,Went for a nice lunch because prices in the Ga...,4.0,2.5,1.0,0.0,0.0,1.0,1.0,...,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204
108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,Ate dinner there tonight. Service was pretty f...,4.0,3.5,1.0,0.0,0.0,0.0,0.0,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,This is the best Japanese food I've had in Nas...,5.0,4.0,1.0,0.0,0.0,1.0,0.0,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,Ok so this is the best Chinese food I've had i...,5.0,3.5,1.0,0.0,1.0,1.0,0.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.039897,0.000385
331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,I love this place I got my lashes done by Nina...,5.0,3.5,0.0,0.0,0.0,0.0,0.0,...,0.001111,0.001111,0.223333,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87906,hn3Rg2JrhQoDJBEhrpuwWg,zzz-M4QvkEpUWWPL9RTzLA,We have been twice and LOVED it the first time...,2.0,3.5,1.0,1.0,0.0,1.0,1.0,...,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.021158,0.010656,0.000105
153153,Y989Hsw30AYUMcyNjBz9iQ,zzz-M4QvkEpUWWPL9RTzLA,We went here last Friday night with friends an...,2.0,3.5,1.0,1.0,0.0,0.0,1.0,...,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.032951,0.000164,0.000164
153155,dNmqRPHBys4dWH4nReO7MA,zzz-M4QvkEpUWWPL9RTzLA,"Based on the website, I was expecting somethin...",1.0,3.5,1.0,1.0,1.0,0.0,1.0,...,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370
153392,qO-r-NipYXcuVXjIbaglEQ,zzz-M4QvkEpUWWPL9RTzLA,I got my haircut on Saturday and I've had at l...,4.0,4.0,0.0,0.0,0.0,0.0,0.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385


In [23]:
# LDA调参
from sklearn.model_selection import GridSearchCV
parameters = {'learning_method': 'online', 
              'n_components': range(100, 200, 50),
              'learning_offset': range(50, 101, 50)}
lda = LatentDirichletAllocation()
model = GridSearchCV(lda, parameters)
model.fit(cntTf)

sorted(model.cv_results_.keys())


KeyboardInterrupt: 

In [12]:
# testing set估计topic distribution
user_groups = dict(list(all_train.groupby('user_id')))
business_groups = dict(list(all_train.groupby('business_id')))

user_profile = {}
for user, df in user_groups.items():
    p_hat = np.mean(np.array(df.iloc[:,14:]), axis=0)
    p = p_hat / np.sum(p_hat)
    user_profile[user] = p
business_profile = {}
for business, df in business_groups.items():
    q_hat = np.mean(np.array(df.iloc[:,14:]), axis=0)
    q = q_hat / np.sum(q_hat)
    business_profile[business] = q

probs = []
for i in range(all_test.shape[0]):
    try:
        user = all_test.iloc[i, 1]
        business = all_test.iloc[i, 0]
        theta = user_profile[user] * business_profile[business]
        theta = theta / np.sum(theta)
        probs.append(theta)
    except:
        theta = np.array([np.nan]*100)
        probs.append(theta)
        
probs = np.array(probs)

for i in range(100):
    name = 'topic_' + str(i+1)
    all_test[name] = probs[:,i]
    
all_test = all_test.dropna(axis=0,how='any')
all_test

,business_id,user_id,text,stars,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,The place is absolutely huge! That's the attra...,4.0,3.0,1.0,1.0,0.0,1.0,1.0,...,0.000012,0.000027,0.000005,9.320489e-07,0.000043,9.320489e-07,0.000002,0.000013,0.000032,0.000012
499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,Ok guys this is one of my favorite vegan spots...,5.0,4.5,1.0,1.0,1.0,1.0,0.0,...,0.098289,0.000005,0.000791,4.591333e-06,0.000017,4.591333e-06,0.001446,0.000005,0.000943,0.000019
533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,I love this place I tried the Cuban sandwich a...,5.0,4.5,0.0,1.0,0.0,0.0,0.0,...,0.027581,0.000005,0.000386,4.700108e-06,0.000059,4.700108e-06,0.000846,0.000042,0.001169,0.000064
190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,It's greasy but what do you expect. It is good...,3.0,4.0,1.0,1.0,0.0,1.0,1.0,...,0.001919,0.000024,0.000013,5.558423e-06,0.000086,5.577801e-06,0.000010,0.000086,0.000042,0.001630
394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,Definitely gets busy so reserve spot in line o...,5.0,4.0,1.0,1.0,0.0,0.0,0.0,...,0.001160,0.000020,0.000016,4.482868e-06,0.000015,4.482868e-06,0.000009,0.000050,0.000113,0.001357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357236,d59JUaCVMZj-ZEtlWcOKZw,zzsPf8xNW11nd0B6MZqfRw,Awesome unique food and a great addition to do...,5.0,4.5,0.0,1.0,1.0,0.0,0.0,...,0.002136,0.000068,0.000003,2.980672e-06,0.000547,2.980672e-06,0.000003,0.000357,0.000003,0.000112
335253,nKJ6RnjIVfWrXO2DUrBgCw,zzz-M4QvkEpUWWPL9RTzLA,I ordered the fish tacos the other night and t...,1.0,3.0,1.0,1.0,0.0,0.0,1.0,...,0.000019,0.000194,0.000001,1.300730e-06,0.000003,1.300730e-06,0.000001,0.000475,0.000341,0.000087
370287,8BEIpKZu-ddcptxDbqMwLQ,zzz-M4QvkEpUWWPL9RTzLA,"I went here for the first time this morning, a...",4.0,3.5,1.0,1.0,1.0,0.0,0.0,...,0.000027,0.000141,0.000002,1.847042e-06,0.000005,1.847042e-06,0.000002,0.001541,0.001171,0.000219
390130,8WNQxb0CiROek9SW7PopDQ,zzz-M4QvkEpUWWPL9RTzLA,What a horrible experience. Either the waitres...,1.0,4.0,1.0,1.0,0.0,1.0,1.0,...,0.000078,0.000050,0.000003,1.627046e-06,0.000030,1.627046e-06,0.000002,0.002493,0.000537,0.000104


In [13]:
all_train = all_train.drop(columns=['text'])
all_test = all_test.drop(columns=['text'])
all_train = all_train.drop(columns=['stars'])
all_test = all_test.drop(columns=['stars'])

In [14]:
all_train

,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
108712,arQfMJal1tl67Z96ROgPFg,--Kwhcbkh7jxkhVVQZo2uQ,2.5,1.0,0.0,0.0,1.0,1.0,0.0,1.0,...,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204,0.000204
108739,tIvfmgT1qMeAEQf8CI5fPQ,--Kwhcbkh7jxkhVVQZo2uQ,3.5,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189,0.000189
300758,65_MIvrauhGs_jssN4-uUQ,--RUqD8c6ibQSh9owooVxw,4.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526,0.000526
318212,xxx_0Ol7lbK0hVzv7x0-Yg,--RUqD8c6ibQSh9owooVxw,3.5,1.0,0.0,1.0,1.0,0.0,1.0,1.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.039897,0.000385
331053,UesqLOoyOGPQsonHpMb59g,--RUqD8c6ibQSh9owooVxw,3.5,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.001111,0.001111,0.223333,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111,0.001111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87906,hn3Rg2JrhQoDJBEhrpuwWg,zzz-M4QvkEpUWWPL9RTzLA,3.5,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.000105,0.021158,0.010656,0.000105
153153,Y989Hsw30AYUMcyNjBz9iQ,zzz-M4QvkEpUWWPL9RTzLA,3.5,1.0,1.0,0.0,0.0,1.0,1.0,1.0,...,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.000164,0.032951,0.000164,0.000164
153155,dNmqRPHBys4dWH4nReO7MA,zzz-M4QvkEpUWWPL9RTzLA,3.5,1.0,1.0,1.0,0.0,1.0,1.0,1.0,...,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370,0.000370
153392,qO-r-NipYXcuVXjIbaglEQ,zzz-M4QvkEpUWWPL9RTzLA,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385,0.000385


In [15]:
all_test

,business_id,user_id,Ratings,RestaurantsGoodForGroups,OutdoorSeating,DogsAllowed,RestaurantsDelivery,HappyHour,WheelchairAccessible,BikeParking,...,topic_91,topic_92,topic_93,topic_94,topic_95,topic_96,topic_97,topic_98,topic_99,topic_100
108914,ORL4JE6tz3rJxVqkdKfegA,--Kwhcbkh7jxkhVVQZo2uQ,3.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,...,0.000012,0.000027,0.000005,9.320489e-07,0.000043,9.320489e-07,0.000002,0.000013,0.000032,0.000012
499384,QRzaTTTE7QrqjW2GOQIrtA,--RUqD8c6ibQSh9owooVxw,4.5,1.0,1.0,1.0,1.0,0.0,0.0,1.0,...,0.098289,0.000005,0.000791,4.591333e-06,0.000017,4.591333e-06,0.001446,0.000005,0.000943,0.000019
533424,rs2OZH_vIKgW2Zk59OnZ2A,--RUqD8c6ibQSh9owooVxw,4.5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.027581,0.000005,0.000386,4.700108e-06,0.000059,4.700108e-06,0.000846,0.000042,0.001169,0.000064
190762,VAeEXLbEcI9Emt9KGYq9aA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,...,0.001919,0.000024,0.000013,5.558423e-06,0.000086,5.577801e-06,0.000010,0.000086,0.000042,0.001630
394742,VHw4q0f8WmajCUD2wGXBDA,--SQl1BQaK7Sjpt5ZZGStw,4.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.001160,0.000020,0.000016,4.482868e-06,0.000015,4.482868e-06,0.000009,0.000050,0.000113,0.001357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
357236,d59JUaCVMZj-ZEtlWcOKZw,zzsPf8xNW11nd0B6MZqfRw,4.5,0.0,1.0,1.0,0.0,0.0,1.0,1.0,...,0.002136,0.000068,0.000003,2.980672e-06,0.000547,2.980672e-06,0.000003,0.000357,0.000003,0.000112
335253,nKJ6RnjIVfWrXO2DUrBgCw,zzz-M4QvkEpUWWPL9RTzLA,3.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.000019,0.000194,0.000001,1.300730e-06,0.000003,1.300730e-06,0.000001,0.000475,0.000341,0.000087
370287,8BEIpKZu-ddcptxDbqMwLQ,zzz-M4QvkEpUWWPL9RTzLA,3.5,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.000027,0.000141,0.000002,1.847042e-06,0.000005,1.847042e-06,0.000002,0.001541,0.001171,0.000219
390130,8WNQxb0CiROek9SW7PopDQ,zzz-M4QvkEpUWWPL9RTzLA,4.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,0.000078,0.000050,0.000003,1.627046e-06,0.000030,1.627046e-06,0.000002,0.002493,0.000537,0.000104


In [16]:
all_train.to_csv('train_data/all_train_100.csv')
all_test.to_csv('train_data/all_test_100.csv')